# Clustering beats and fills on Expanded Groove Midi Dataset
*https://magenta.tensorflow.org/datasets/e-gmd*

In [ ]:
import sys
import os
parent_dir = os.path.abspath(os.path.join('..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
%load_ext autoreload
%autoreload 2

import mirdata
import librosa
import itertools
import umap
import umap.plot
import numpy as np
import pandas as pd
from scripts.data_loaders import load_malian_jembe_dataset, load_candombe_dataset, load_cretan_dances_dataset, load_ballroom_dataset
from sklearn.preprocessing import LabelEncoder
from scripts.scale_transform_magnitude import compute_stm
from scripts.clusterers import select_best_num_clusters
from pathlib import Path
from tqdm import tqdm

In [ ]:
# clean metadata
metadata = pd.read_csv(
    "../datasets/e-gmd-v1.0.0/e-gmd-v1.0.0.csv"
)  # read metadata of extended groove midi

metadata_beats = metadata[metadata["beat_type"] == "beat"]  # select only beats

metadata_beats = metadata_beats[metadata_beats["duration"] > 30]
print("duration summary in seconds: \n", metadata_beats["duration"].describe())

metadata_beats["genre"] = metadata_beats["style"].apply(
    lambda x: x.split("/")[0]
)  # create genre column based on style

metadata_beats = metadata_beats[["genre", "style", "audio_filename", "duration"]] # get rid of unnecessary columns
metadata_beats = metadata_beats.reset_index(drop=True) # resetting index is neede for interactive plot

metadata_beats

In [ ]:
# preparing the data and computing stm
groove_midi_path = Path("../datasets/e-gmd-v1.0.0")
features = []
for row in tqdm(metadata_beats.itertuples(index=False), total=metadata_beats.shape[0]): # TODO: find a more efficient way to loop
    try:
        # TODO: segment audio file?
        y, sr = librosa.load(groove_midi_path / row.audio_filename, sr=None, duration=30)
        features.append(
            compute_stm(y=y, sr=sr)[:200]
        )  # discarding coefficients at the end
    except Exception as e:
        print(f"Error: {e}")

### K-Means clustering and Silhouette analysis
*https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py*

The plot on the left-hand side displays the silhouette score. The silhouette score measures how similar an object is to its own cluster compared to other clusters. 

A high silhouette score indicates that clusters are well-separated, while a low score suggests overlapping clusters or misclassification.

From the thickness of the silhouette plot the cluster size can be visualized.


In [ ]:
num_of_clusters = [i for i in range(3, 8)]
results, optimal_k = select_best_num_clusters(
    n_clusters=num_of_clusters, X=np.array(features), dim_reduction="tsne", cluster_method="kmedoids"
)

print(f"Best number of clusters: {optimal_k}; silhouette score: {results.get(optimal_k)}")

### Visualize data with interactive UMAP plot

*https://umap-learn.readthedocs.io/en/latest/plotting.html#interactive-plotting-and-hover-tools*

In [ ]:
labels = pd.factorize(metadata_beats["genre"])[0]  # integer labels needed for the interactive plot
reducer = umap.UMAP(metric="cosine").fit(features)  # reduce dimensionality

p = umap.plot.interactive(
    reducer, labels=labels, hover_data=metadata_beats, point_size=3,
)  # interactive plot, hover_data can be customized

umap.plot.output_file("groove_midi_beats.html") # save the plot locally
umap.plot.output_notebook() # display inline in notebook
umap.plot.show(p)

<hr>

# Clustering on Candombe, Malian Jembè, GreekDances, Ballroom and Cuban Salsa

In [32]:
# define parameters to compute scale transform magnitude
stm_params = {"mel_flag" : True, "with_padding" : True, "n_mels" : 50, "autocor_window_type" : "hamming", "num_stm_coefs" : 150}

In [33]:
features_mj, labels_mj, hover_data_mj = load_malian_jembe_dataset(stm_params=stm_params)
hover_data_mj

Processing annotation file: MJ_Maraka_1_Annotation.csv
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_1_J1.wav
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_1_J2.wav
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_1_D1.wav
Processing annotation file: MJ_Maraka_3_Annotation.csv
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_3_D1.wav
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_3_J2.wav
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_3_J1.wav
Processing annotation file: MJ_Maraka_2_Annotation.csv
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_2_D1.wav
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_2_J1.wav
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_2_J2.wav
Processing annotation file: MJ_Maraka_6_Annotation.csv
Processing audio file: ../datasets/MJ/Media/MJ_Maraka/MJ_Maraka_6_J2.wav
Processing audio file: ../datasets/MJ/Media/MJ_Mar

,pattern,instrument,tradition,label
0,basic theme 1,J1,Maraka,Maraka
1,basic theme 2,J1,Maraka,Maraka
2,basic theme 3,J1,Maraka,Maraka
3,improvisation 1,J1,Maraka,Maraka
4,theme 2,J1,Maraka,Maraka
...,...,...,...,...
372,basic pattern 1,J1,Manjanin,Manjanin
373,impros,J1,Manjanin,Manjanin
374,heating up,J1,Manjanin,Manjanin
375,impro,J1,Manjanin,Manjanin


In [39]:
features_candombe, labels_candombe, hover_data_candombe = load_candombe_dataset(stm_params=stm_params)
hover_data_candombe

Processing file: UC_241_Chico.wav
Processing file: UC_241_Piano.wav
Processing file: UC_241_Repique.wav
Processing file: UC_211_Repique.wav
Processing file: UC_211_Piano.wav
Processing file: UC_211_Chico.wav
Processing file: UC_231_Repique.wav
Processing file: UC_231_Piano.wav
Processing file: UC_231_Chico.wav
Processing file: UC_213_Chico.wav
Processing file: UC_213_Piano.wav
Processing file: UC_213_Repique.wav
Processing file: UC_312_Chico.wav
Processing file: UC_312_Repique1.wav
Processing file: UC_312_Piano.wav
Processing file: UC_312_Repique2.wav
Processing file: UC_242_Piano.wav
Processing file: UC_242_Chico.wav
Processing file: UC_242_Repique.wav
Processing file: UC_232_Chico.wav
Processing file: UC_232_Piano.wav
Processing file: UC_232_Repique.wav
Processing file: UC_212_Piano.wav
Processing file: UC_212_Repique.wav
Processing file: UC_212_Chico.wav
Processing file: UC_222_Chico.wav
Processing file: UC_222_Piano.wav
Processing file: UC_222_Repique.wav
Processing file: UC_321_Pi

,pattern,instrument,tradition,label
0,0,Chico,Candombe,Chico
1,1,Chico,Candombe,Chico
2,2,Chico,Candombe,Chico
3,3,Chico,Candombe,Chico
4,4,Chico,Candombe,Chico
...,...,...,...,...
321,3,Chico,Candombe,Chico
322,4,Chico,Candombe,Chico
323,5,Chico,Candombe,Chico
324,6,Chico,Candombe,Chico


In [35]:
features_cretan, labels_cretan, hover_data_cretan = load_cretan_dances_dataset(stm_params=stm_params)
hover_data_cretan

Processing folder: maleviziotis
Processing folder: kalamatianos
Processing folder: sousta
Processing folder: kontilies
Processing folder: pentozalis
Processing folder: syrtos


,pattern,instrument,tradition,label
0,07.Malevuziwths_lyr1,None,CretanDances,maleviziotis
1,020_M_A_lyr1,None,CretanDances,maleviziotis
2,13-Maleviziwtis_lyr1,None,CretanDances,maleviziotis
3,12-Malevizwtis_lyr1,None,CretanDances,maleviziotis
4,10-Meleviziwtis_lyr1,None,CretanDances,maleviziotis
...,...,...,...,...
174,01 - st_alargino taksidi soy_lyr1,None,CretanDances,syrtos
175,"13-Nous Taksideuths , X.Stivaktakis_lyr1",None,CretanDances,syrtos
176,044_M_A_lyr1,None,CretanDances,syrtos
177,09 Surto Apokorwniwtiko_lyr1,None,CretanDances,syrtos


In [36]:
features_ballroom, labels_ballroom, hover_data_ballroom = load_ballroom_dataset(stm_params=stm_params)
hover_data_ballroom

Processing folder: Samba
Processing folder: Rumba-Misc
Processing folder: Tango
Processing folder: VienneseWaltz
Processing folder: Quickstep
Processing folder: Waltz
Processing folder: Rumba-American
Processing folder: ChaChaCha
Processing folder: Rumba-International
Processing folder: Jive
Features shape: 698 -- labels shape: 698


,pattern,instrument,tradition,label
0,Media-103607,None,Ballroom,samba
1,Media-106107,None,Ballroom,samba
2,Media-104102,None,Ballroom,samba
3,Media-104001,None,Ballroom,samba
4,Media-103504,None,Ballroom,samba
...,...,...,...,...
693,Media-105414,None,Ballroom,jive
694,Albums-Pais_Tropical-13,None,Ballroom,jive
695,Media-104017,None,Ballroom,jive
696,Media-103919,None,Ballroom,jive


In [37]:
combined_features = list(itertools.chain(features_mj, features_candombe, features_cretan, features_ballroom))
combined_labels = list(itertools.chain(hover_data_mj["label"], hover_data_candombe["label"], hover_data_cretan["label"], hover_data_ballroom["label"]))
combined_hover_data = pd.concat([hover_data_mj, hover_data_candombe, hover_data_cretan, hover_data_ballroom]).reset_index(drop=True)

### K-Means clustering and Silhouette analysis


In [ ]:
num_of_clusters = [i for i in range(3, 5)]
results, optimal_k = select_best_num_clusters(
    n_clusters=num_of_clusters, X=np.array(combined_features), dim_reduction="tsne", cluster_method="kmedoids"
)

print(f"Best number of clusters: {optimal_k}; silhouette score: {results.get(optimal_k)}")

### Visualize data with interactive UMAP plot

In [40]:
encoded_labels = LabelEncoder().fit_transform(combined_labels)  # integer labels needed for the interactive plot
reducer = umap.UMAP(metric="cosine").fit(combined_features) # https://umap-learn.readthedocs.io/en/latest/parameters.html#basic-umap-parameters

p = umap.plot.interactive(
    reducer, labels=encoded_labels, hover_data=combined_hover_data, point_size=5,
)  # interactive plot, hover_data can be customized

# umap.plot.output_file("mj.html") # save the plot locally
umap.plot.output_notebook() # display inline in notebook
umap.plot.show(p)

Loading BokehJS ...